# **Code for first presentation**

In [1]:
import gym
import slimevolleygym
from gym import wrappers
import time
from matplotlib import animation
import matplotlib.pyplot as plt
env_to_wrap = gym.make("SlimeVolley-v0")

/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
env = wrappers.Monitor(env_to_wrap, './videos/SlimeVolley' + time.strftime('%Y_%m_%d_%H_%S') + '/', force = True)
state = env.reset()
t=0
while True:
    t+=1
    env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()
env_to_wrap.close()

Episode finished after 738 timesteps


In [7]:
env = gym.make("SlimeVolley-v0")
env.unwrapped.get_action_meanings()

['NOOP', 'UP', 'RIGHT', 'LEFT', 'UPRIGHT', 'UPLEFT']

# **Second part of the project**

In [3]:
#python version must be 3.7
!pip install tensorflow==1.15

1.15.0


In [4]:
!pip install stable-baselines[mpi]==2.10.2
!pip install slimevolleygym

  Using cached stable_baselines-2.10.2-py3-none-any.whl (240 kB)
     |████████████████████████████████| 38.1 MB 157 kB/s eta 0:00:01     |████████▋                       | 10.2 MB 1.2 MB/s eta 0:00:23
  Using cached cloudpickle-2.0.0-py3-none-any.whl (25 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
     |████████████████████████████████| 11.3 MB 1.7 MB/s eta 0:00:01
     |████████████████████████████████| 60.3 MB 48 kB/s  eta 0:00:01     |████████▍                       | 15.8 MB 2.9 MB/s eta 0:00:16     |█████████                       | 17.0 MB 1.5 MB/s eta 0:00:29     |██████████████████▏             | 34.1 MB 1.9 MB/s eta 0:00:14     |████████████████████████████    | 52.9 MB 3.0 MB/s eta 0:00:03
  Using cached gym-0.21.0.tar.gz (1.5 MB)
     |████████████████████████████████| 11.2 MB 2.4 MB/s eta 0:00:01    |██████████████▎                 | 5.0 MB 3.3 MB/s eta 0:00:02
  Using cached mpi4py-3.1.3.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting 

### PPO1 algorithm directly trained against Built-in expert agent

In [3]:
import gym
import slimevolleygym
from stable_baselines import PPO1
from stable_baselines.common.evaluation import evaluate_policy
from stable_baselines.common.callbacks import EvalCallback
from stable_baselines.bench import Monitor

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [4]:
env = gym.make('SlimeVolley-v0')
env = Monitor(env, log_dir)
eval_env = gym.make('SlimeVolley-v0')

eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=False, render=False)

model = PPO1('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=10000, callback=eval_callback)

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

NameError: name 'log_dir' is not defined

### Video

In [16]:
from gym import wrappers
import time
from matplotlib import animation
import matplotlib.pyplot as plt

In [21]:
env = wrappers.Monitor(eval_env, './videos/SlimeVolley' + time.strftime('%Y_%m_%d_%H_%M') + '_PPO' +'/', force = True)
state = env.reset()
t=0
while True:
    t+=1
    env.render()
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()
eval_env.close()

Episode finished after 608 timesteps


### PPO2 algorithm directly trained against Built-in expert agent

In [ ]:
from stable_baselines import PPO2
env = gym.make('SlimeVolley-v0')

eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=False, render=False)

model = PPO2('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=10000, callback=eval_callback)

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)


### Optimized PPO1 algorithm directly trained against Built-in expert agent

In [ ]:
env = gym.make('SlimeVolley-v0')
eval_env = gym.make('SlimeVolley-v0')

eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=10000,
                             deterministic=False, render=False)

model_1 = PPO1('MlpPolicy', env, timesteps_per_actorbatch=4096, clip_param=0.2, entcoeff=0.0
               , optim_epochs=10,optim_stepsize=3e-4, optim_batchsize=64, gamma=0.99
               , lam=0.95, schedule='linear', verbose=2)
model_1.learn(total_timesteps=1000000, callback=eval_callback)
model_1.save(os.path.join('models', "PPO_model"))

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model_1, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

### Video

In [ ]:
env = wrappers.Monitor(eval_env, './videos/SlimeVolley' + time.strftime('%Y_%m_%d_%H_%M') + '_PPO' +'/', force = True)
state = env.reset()
t=0
while True:
    t+=1
    env.render()
    action, _states = model_1.predict(state)
    state, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()
eval_env.close()

## **A2C directly traind against Built-in expert againt**

In [4]:
from stable_baselines import A2C

In [5]:
env = gym.make('SlimeVolley-v0')
eval_env = gym.make('SlimeVolley-v0')

eval_callback = EvalCallback(eval_env, best_model_save_path='./logs/',
                             log_path='./logs/', eval_freq=500,
                             deterministic=False, render=False)

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000, callback=eval_callback)

# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Wrapping the env in a DummyVecEnv.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

---------------------------------------
| EpLenMean          | 631            |
| EpRewMean          | -4.69          |
| EpThisIter         | 1              |
| EpisodesSoFar      | 16             |
| TimeElapsed        | 74.1           |
| TimestepsSoFar     | 10240          |
| ev_tdlam_before    | 0.888          |
| explained_variance | -145           |
| fps                | 13             |
| loss_ent           | 1.9290515      |
| loss_kl            | 3.0267984e-08  |
| loss_pol_entpen    | -0.019290514   |
| loss_pol_surr      | -4.6381727e-05 |
| loss_vf_loss       | 0.005139376    |
| nupdates           | 1              |
| policy_entropy     | 2.08           |
| total_timesteps    | 5              |
| value_loss         | 0.0156         |
---------------------------------------
Eval num_timesteps=500, episode_reward=-5.00 

Eval num_timesteps=9500, episode_reward=-4.60 +/- 0.49
Episode length: 605.00 +/- 80.07
---------------------------------
| explained_variance | 0.948    |
| fps                | 142      |
| nupdates           | 1900     |
| policy_entropy     | 2.03     |
| total_timesteps    | 9500     |
| value_loss         | 0.000208 |
---------------------------------
Eval num_timesteps=10000, episode_reward=-4.80 +/- 0.40
Episode length: 625.00 +/- 85.11
---------------------------------
| explained_variance | -2.85    |
| fps                | 141      |
| nupdates           | 2000     |
| policy_entropy     | 2.06     |
| total_timesteps    | 10000    |
| value_loss         | 0.00386  |
---------------------------------
mean_reward:-4.82 +/- 0.48


## Video

In [24]:
env = wrappers.Monitor(eval_env, './videos/SlimeVolley' + time.strftime('%Y_%m_%d_%H_%M') + '_A2C' +'/', force = True)
state = env.reset()
t=0
while True:
    t+=1
    env.render()
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)
    if done:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()
eval_env.close()

Episode finished after 428 timesteps


### Note:
I have used the stabebaseline built-in ploter from the link below:
https://colab.research.google.com/github/Stable-Baselines-Team/rl-colab-notebooks/blob/master/monitor_training.ipynb#scrollTo=BIedd7Pz9sOs